# Técnicas de Score de modelos en Python

Las técnicas de Score (o puntuación) de modelos de ML evalúan su precisión. Habitualmente se basan en realizar algún tipo de comparación entre la salida obtenida y la salida deseada.

Hay muchas técnicas de Score en ML, pero una familia de técnicas muy populares son las basadas en las Confusion matrices. Estas matrices cruzan las predicciones posibles, con las salidas deseadas, estableciendo una clasificación de la salida del modelo como:
- TN: True Negative, el modelo predice correctamente un resultado negativo.
- FN: False Negative, el modelo predice incorrectamente un resultado negativo.
- TP: True Positive, el modelo predice correctamente un resultado positivo.
- FP: False Positive, el modelo predice incorrectamente un resultado positivo.

Las técnicas más comunes populares de este tipo son: precision, recall, f1-score y accuracy.

En Pytorch no se implementa nativamente ninguna técnica de Score. Pero, en el ecosistema de Pytorch, hay un librería estándar de-facto que se puede usar para ello: *torchmetrics*

Este librería tiene un API que implementa varias métricas con una interfaz común que proporciona los métodos: *update()*, *compute()* y *reset()*. Veamos un ejemplo completo de su uso:

In [1]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 10.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
import torch.nn as nn
from torchmetrics import F1Score
from torch.utils.data import DataLoader, TensorDataset

# Definimos un modelo muy simple para clasificación binaria
class BinaryClassificationModel(nn.Module):
    def __init__(self):
        super(BinaryClassificationModel, self).__init__()
        self.linear = nn.Linear(10, 1)
    
    def forward(self, x):
        output = torch.sigmoid(self.linear(x))
        return output

# Inicializamos el modelo, la métrica y el optimizador
model = BinaryClassificationModel()
metric = F1Score(task='binary',num_classes=1, threshold=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Creamos un dataset de ejemplo aleatorio
inputs = torch.randn(100, 10)
targets = torch.randint(0, 2, (100,)).float()  # Binary targets
dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=10)

# Bucle de entrenamiento
for epoch in range(5):  # 5 épocas
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = nn.BCELoss()(outputs.view(-1), labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Actualizamos la métrica
        metric(outputs.view(-1), labels)
        
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")
    print(f"Epoch {epoch+1}, F1-Score: {metric.compute()}")

# Reinicializamos la métrica para la siguiente época
metric.reset()


Epoch 1, Loss: 0.7850720107555389
Epoch 1, F1-Score: 0.4954128563404083
Epoch 2, Loss: 0.7814775586128235
Epoch 2, F1-Score: 0.5023255944252014
Epoch 3, Loss: 0.778220784664154
Epoch 3, F1-Score: 0.5046728849411011
Epoch 4, Loss: 0.7750377476215362
Epoch 4, F1-Score: 0.5058547854423523
Epoch 5, Loss: 0.7719135165214539
Epoch 5, F1-Score: 0.5075187683105469


También podemos usar una estructura de tipo *MetricCollection* para recopilar dos métricas a la vez. Veamos una adaptación del ejemplo anterior

In [3]:
from torchmetrics import HammingDistance, F1Score, MetricCollection


# Inicializamos el modelo, la métrica y el optimizador
model = BinaryClassificationModel()
# Creamos una colección de métricas
# ESCRIBE TU CÓDIGO AQUÍ
# metric = ...
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Creamos un dataset de ejemplo aleatorio
inputs = torch.randn(100, 10)
targets = torch.randint(0, 2, (100,)).long()  # Binary targets
dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=10)

# Bucle de entrenamiento
for epoch in range(5):  # 5 épocas
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = nn.BCELoss()(outputs.view(-1), labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # Actualizamos la métrica
        pred_labels = torch.round(outputs.view(-1))
        # Llamamos a la colección de métricas
        # ESCRIBE TU CÓDIGO AQUÍ
        # metric(...)

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")
    print(f"Epoch {epoch+1}, Metrics: {metrics.compute()}")
    # Reinicializamos la métrica para la siguiente época
    # ESCRIBE TU CÓDIGO AQUÍ
    # metric. ... 

Epoch 1, Loss: 0.799572902917862


NameError: name 'metrics' is not defined

# Ejercicio

Utilizando como el base, el código resultante del ejercicio al final del notebook *optimizers.ipynb*, haz los cambios pertinentes para calcular la Hamming Distance y el BinaryF1Score.

In [ ]:
# Import libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from torchmetrics import HammingDistance, F1Score, MetricCollection


# Inicializamos el modelo, la métrica y el optimizador
metrics = MetricCollection([HammingDistance(task='multiclass', num_classes=10), F1Score(task='multiclass',num_classes=10, threshold=0.5)])

# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Define the network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the loss function
loss_func = nn.CrossEntropyLoss()

# Define the training function
def train(net, trainloader, optimizer, num_epochs=2):
    loss_values = []
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()

            # save losses to plot later
            running_loss += loss.item()
            if i % 2000 == 1999:  # print every 2000 mini-batches
                loss_values.append(running_loss / 2000)
                running_loss = 0.0
                metrics(torch.argmax(outputs,dim=1), labels)
        
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(dataloader)}")
        print(f"Epoch {epoch+1}, Metrics: {metrics.compute()}")
        # Reinicializamos la métrica para la siguiente época
        metrics.reset()

    return loss_values

# SGD
net_SGD = Net()
optimizer_SGD = optim.SGD(net_SGD.parameters(), lr=0.001, momentum=0.9)
loss_SGD = train(net_SGD, trainloader, optimizer_SGD)

